# Histórico e Análise de Experimentos de Classificação de Intenções de Busca

Este *notebook* tem como objetivo permitir o registro visual dos experimentos feitos e oferecer um modo de recuperar um experimento e analisar os resultados em mais detalhes.

## Bibliotecas e Funções

In [1]:
# General
import sys
import funcy as fp
from pathlib import Path

# Visualization / Presentation
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.display import HTML, display

import mlflow
from mlflow.tracking import MlflowClient
import pandas as pd

# Carregar, além de atualizar frequentemente, código personalizado disponível em ../src
%load_ext autoreload 
%autoreload 2
sys.path.append(str(Path.cwd().parent))
from src import settings
from src.utils.notebooks import display_side_by_side
from src.pipeline.inference_pipeline import load_model_resources

# Configurações para a exibição de conteúdo do Pandas e das bibliotecas gráficas
%matplotlib inline 
sns.set(rc={'figure.figsize':(25,10)})
pd.set_option('display.max_rows', None)
pd.set_option("display.max_columns", None)
pd.set_option('max_colwidth', 150)

## Recuperação do melhor resultado

Considerando os experimentos feitos no notebook [Classificação de Intenções](04.3_Classificacao_de_Intencoes.ipynb), faz-se a recuperação dos melhores resultados de cada algoritmo, tendo como métrica principal o F1.

In [2]:
EXPERIMENT_ID = '4'

mlflow_client = MlflowClient()

best_experiments_result = [
    mlflow.search_runs(experiment_ids=[experiment_id], 
                       max_results=100, 
                       order_by=['metrics.F1 DESC'], 
                       filter_string='attributes.status="FINISHED"')
    for experiment_id in [EXPERIMENT_ID]
]

best_results = pd.concat(best_experiments_result, axis=0)

O melhores resultados recuperados:

In [3]:
columns_to_show = ['experiment_name', 'tags.mlflow.runName', 'run_id', 'experiment_id', 'params.model_name',
                   'metrics.f1', 'metrics.precision', 'metrics.recall', 'metrics.auc', 'metrics.mc', 'metrics.training_time']

(best_results
 .assign(experiment_name=lambda f: f['experiment_id'].apply(lambda id: mlflow_client.get_experiment(id).name))
 .sort_values(by='metrics.f1', ascending=False)
 .loc[lambda f: ~f['params.model_name'].isna()]
 .drop_duplicates('params.model_name')
 [columns_to_show]  
 .head()
)

,experiment_name,tags.mlflow.runName,run_id,experiment_id,params.model_name,metrics.f1,metrics.precision,metrics.recall,metrics.auc,metrics.mc,metrics.training_time
7,04_SupervisedQueryIntentClassification,01_7_Embeddings and Artisanal Feaures_CB,3ff29c0d3a524d9c95612cb0d6de54ab,4,CB,0.565854,0.563107,0.568627,0.719285,0.437068,3.430026
31,04_SupervisedQueryIntentClassification,01_9_Artisanal Feaures_RF,700237d7c7b3402fa76068e12958c68e,4,RF,0.476190,0.400000,0.588235,0.664060,0.291552,0.616521
85,04_SupervisedQueryIntentClassification,01_2_Embeddings_MLP,e76250a177c44c5d97e964e05b66334a,4,MLP,0.475000,0.655172,0.372549,0.657456,0.393416,1.684383


## Restauração de Experimentos

A partir da escolha de uma execução individual, é possível restaurar os elementos utilizados na experimentação para aplicá-los aos dados.

In [6]:
RUN_ID = '3ff29c0d3a524d9c95612cb0d6de54ab'

preprocessing_model, model, label_encoder = load_model_resources(RUN_ID)

Para validar o funcionamento da restauração do modelo, parte dos dados de treinamento são recuperados para uma avaliação.

In [7]:
columns_to_read = ['query', 'intent_class', 'intent_description']

frame = pd.read_csv(Path(settings.DATA_PATH).joinpath('interim', 'query_intent_training.csv'), usecols=columns_to_read)

display_side_by_side([frame.head(10)], ['Conjunto de Dados de Intenções de Busca'])

,query,intent_class,intent_description
0,lembrancinha 1 ano,0,Busca Exploratória
1,pioneiros,0,Busca Exploratória
2,fibra siliconada,0,Busca Exploratória
3,vestido 15 anos,0,Busca Exploratória
4,quadros personalizados com fotos,1,Busca Focada
5,festa de 50 anos,1,Busca Focada
6,brindes de final de ano,0,Busca Exploratória
7,porta cartao de vacina,0,Busca Exploratória
8,letras de nomes personalizados,1,Busca Focada
9,futon,0,Busca Exploratória


Tendo os dados, é possível refazer a preparação e fazer a predição da intenção de busca de cada elemento.

In [8]:
frame_slice = frame.sample(50)

# Processa os dados para inferência
features = preprocessing_model.predict(frame_slice)

# Realiza a inferência
frame_slice['pred'] = model.predict(features)

display_side_by_side([frame_slice, 
                      pd.DataFrame(features).describe().T.head(10)], 
                     ['Dados Recuperados e Predição', 
                      f'Features (10 de {features.shape[1]})'])

del frame_slice

,query,intent_class,intent_description,pred
2901,squeeze de plastico personalizado,0,Busca Exploratória,0
2102,lacos,0,Busca Exploratória,0
358,pote de vidro para lembrancinha,0,Busca Exploratória,0
3555,adesivo jateado para vidro,0,Busca Exploratória,0
3715,jogo de cozinha com 3 pecas,0,Busca Exploratória,0
1773,tapete cozinha,0,Busca Exploratória,0
834,festa pokemon,0,Busca Exploratória,0
2216,bloco de anotacoes,0,Busca Exploratória,0
2361,placas,0,Busca Exploratória,0
968,presente para avos,0,Busca Exploratória,0
